In [1]:
import random
import numpy as np
import torch

# Устанавливаем фиксированный сид
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # Для всех устройств CUDA

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
import sys
import os
from pathlib import Path

# Добавляем путь на уровень выше
sys.path.append(str(Path(os.getcwd()).resolve().parent))

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from utils.load_data import load_data
from utils.features import *
from utils.feature_engineering import add_features
from utils.training import run_grid_search
from utils.data_processing import normalize_data, preprocess_data_multichannel
from models.tgcn import TGCN, TGCNEmb
from models.gru import GRUModel, GRUModelEmb
from models.tcn import TCNModel, TCNModelEmb
from models.dfdgcn import DFDGCN
from models.lstm import LSTMModel
from models.dcrnn import DCRNNModel
from models.stconv import STConvModel

datasets_metadata = {
    'METR-LA': {'start_date': '2012-03-01', 'frequency': 5},
    'PEMS-BAY': {'start_date': '2017-01-01', 'frequency': 5},
    'PEMS03': {'start_date': '2018-01-09', 'frequency': 5},
    'PEMS04': {'start_date': '2018-01-01', 'frequency': 5},
    'PEMS07': {'start_date': '2012-05-01', 'frequency': 5},
    'PEMS08': {'start_date': '2018-07-01', 'frequency': 5},
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_dir = '../data/all_data/PEMS-BAY'
data, metadata, adj_matrix = load_data(data_dir)
data = data[:2016, :, 0]

In [4]:
# Преобразование в тензоры PyTorch
A = torch.tensor(adj_matrix.values, dtype=torch.float32)
edge_index = A.nonzero(as_tuple=False).t().to(device)

In [5]:
print(f"max(edge_index): {edge_index.max()}")
print(f"min(edge_index): {edge_index.min()}")
print(f"num_nodes: {adj_matrix.shape[0]}")


max(edge_index): 324
min(edge_index): 0
num_nodes: 325


In [6]:
# Предобработка данных
time_len = len(data)
rate = 0.8
seq_len = 12
pre_len = 6
train_len = int(time_len * rate)

# Разделение данных на train и test
train_data = data[:train_len]
test_data = data[train_len:]
index_train = pd.date_range(
    start=datasets_metadata[metadata['name']]['start_date'],
    periods=train_len,
    freq=f'{datasets_metadata[metadata["name"]]["frequency"]}T'
)
index_test = pd.date_range(
    start=index_train[-1] + pd.Timedelta(minutes=datasets_metadata[metadata["name"]]["frequency"]),
    periods=len(test_data),
    freq=f'{datasets_metadata[metadata["name"]]["frequency"]}T'
)

E:\Temp\ipykernel_20236\3312065895.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  index_train = pd.date_range(
E:\Temp\ipykernel_20236\3312065895.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  index_test = pd.date_range(


In [7]:
# Расширение размерности для train и test
train_data_extended = train_data[..., np.newaxis]
test_data_extended = test_data[..., np.newaxis]

# Определение функций
feature_functions = {
    'min': minute_index_feature,
    # 'hour': hour_feature,
    'weekday': weekday_index_feature,
    # 'peak_hours': peak_hours_feature,
    # 'mean': mean_feature,
    # 'median': median_feature,
    # 'std': std_feature,
    # 'min': min_feature,
    # 'max': max_feature,
    # 'kurtosis': kurtosis_feature,
    # 'skew': skew_feature,
    # 'quantile': quantile_feature,
    # 'rolling_mean': rolling_mean_feature,
    # 'rolling_std': rolling_std_feature,
    # 'rolling_min': rolling_min_feature,
    # 'rolling_max': rolling_max_feature,
}

graph_feature_functions = {
    'degree': degree_feature,
    'degree_centrality': degree_centrality_feature,
    'closeness_centrality': closeness_centrality_feature,
    'betweenness_centrality': betweenness_centrality_feature,
    'clustering_coefficient': clustering_coefficient_feature
}

# Добавление фичей для train
train_data_extended = add_features(
    train_data_extended,
    feature_list=list(feature_functions.keys()),
    feature_functions=feature_functions,
    graph_feature_functions=graph_feature_functions,
    index=index_train,
    # adj_matrix=adj_matrix
)

# Добавление фичей для test
test_data_extended = add_features(
    test_data_extended,
    feature_list=list(feature_functions.keys()),
    feature_functions=feature_functions,
    graph_feature_functions=graph_feature_functions,
    index=index_test,
    # adj_matrix=adj_matrix
)

In [ ]:
feature_names = ['Time series'] + list(feature_functions.keys()) + list(graph_feature_functions.keys())

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def corr_show(data, feature_names):
    # 1. Преобразование данных в двумерный формат
    num_timesteps, num_nodes, num_features = data.shape
    data_2d = data.reshape(-1, num_features)  # Форма: (2016 * 2, C)

    # 2. Создание DataFrame для удобства
    df = pd.DataFrame(data_2d, columns=feature_names)

    # 3. Вычисление матрицы корреляции
    correlation_matrix = df.corr()
    # correlation_matrix.dropna(how='all', inplace=True)
    # correlation_matrix.dropna(axis=1, how='all', inplace=True)

    # 4. Визуализация матрицы корреляции
    plt.figure(figsize=(16, 14))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation Matrix of Features')
    plt.show()

corr_show(train_data_extended, feature_names)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def apply_pca_to_channels(data, channel_start, channel_end, pca_variance=0.95, standardize=True):
    """
    Применяет PCA к указанным каналам данных и заменяет их на новые компоненты.

    Параметры:
        data (np.array): Исходные данные размерностью (samples, time_steps, features).
        channel_start (int): Начальный индекс каналов для обработки.
        channel_end (int): Конечный индекс каналов для обработки.
        pca_variance (float): Доля дисперсии, которую нужно сохранить (по умолчанию 0.95).
        standardize (bool): Применять ли стандартизацию перед PCA (по умолчанию True).

    Возвращает:
        data_new (np.array): Данные с замененными каналами.
        pca: Объект PCA.
        scaler: Объект StandardScaler (если стандартизация применялась).
    """
    # Выделяем указанные каналы
    channels = data[:, :, channel_start:channel_end]  # Размерность (samples, time_steps, n_channels)

    # Преобразуем данные в (samples * time_steps, n_channels)
    data_reshaped = channels.reshape(-1, channels.shape[-1])

    # Стандартизация (если включена)
    scaler = None
    if standardize:
        scaler = StandardScaler()
        data_reshaped_scaled = scaler.fit_transform(data_reshaped)
    else:
        data_reshaped_scaled = data_reshaped

    # Применяем PCA
    pca = PCA(n_components=pca_variance)  # Сохраняем указанную долю дисперсии
    data_pca = pca.fit_transform(data_reshaped_scaled)

    # Преобразуем данные обратно в (samples, time_steps, n_components)
    n_components = data_pca.shape[1]  # Количество компонент после PCA
    data_pca_reshaped = data_pca.reshape(data.shape[0], data.shape[1], n_components)

    # Заменяем каналы в исходном массиве на данные после PCA
    data_new = data.copy()  # Создаем копию исходного массива
    data_new[:, :, channel_start:channel_start + n_components] = data_pca_reshaped

    # Если количество компонент меньше, чем исходное количество каналов, удаляем оставшиеся каналы
    if n_components < (channel_end - channel_start):
        data_new = np.delete(data_new, range(channel_start + n_components, channel_end), axis=2)

    # Результаты
    print("Исходная размерность данных:", data.shape)
    print("Исходная размерность каналов:", channels.shape)
    print("Размерность после PCA:", data_pca_reshaped.shape)
    print("Объясненная дисперсия:", pca.explained_variance_ratio_.sum())
    print("Размерность данных после замены:", data_new.shape)
    print()

    return data_new, pca, scaler

# Применяем PCA к каналам с 4 по 16 (индексы 4:16)
train_data_extended_new, pca, scaler = apply_pca_to_channels(
    data=train_data_extended,
    channel_start=4,
    channel_end=16,
    pca_variance=4,
    standardize=False
)

train_data_extended_new, pca, scaler = apply_pca_to_channels(
    data=train_data_extended_new,
    channel_start=7,
    channel_end=12,
    pca_variance=2,
    standardize=False
)

# Применяем PCA к каналам с 4 по 16 (индексы 4:16)
test_data_extended_new, pca, scaler = apply_pca_to_channels(
    data=test_data_extended,
    channel_start=4,
    channel_end=16,
    pca_variance=4,
    standardize=False
)

test_data_extended_new, pca, scaler = apply_pca_to_channels(
    data=test_data_extended_new,
    channel_start=7,
    channel_end=12,
    pca_variance=2,
    standardize=False
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

n_channels = train_data_extended_new.shape[2]
fig, axes = plt.subplots(n_channels, 1, figsize=(10, 4 * n_channels))
if n_channels == 1:
    axes = [axes]

# Построение гистограмм для каждого канала
for i in range(n_channels):
    channel_data = train_data_extended_new[:, :, i].reshape(-1, 1)
    
    sns.histplot(channel_data, kde=True, ax=axes[i])
    axes[i].set_title(f'Channel {i + 1}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')


plt.tight_layout()
plt.show()

In [8]:
scale = True  # Флаг для включения/выключения нормализации

# Обработка train и test для моделей
trainX, trainY = preprocess_data_multichannel(train_data_extended, seq_len, pre_len)
testX, testY = preprocess_data_multichannel(test_data_extended, seq_len, pre_len)

# Нормализация данных (если scale = True)
if scale:
    trainX_norm, testX_norm, scaler_X = normalize_data(trainX, testX, method='norm')
    trainY_norm, testY_norm, scaler_Y = normalize_data(trainY.reshape(-1, 1), testY.reshape(-1, 1), method='norm')
    trainY_norm = trainY_norm.reshape(trainY.shape)
    testY_norm = testY_norm.reshape(testY.shape)
else:
    # Если нормализация отключена, используем исходные данные
    trainX_norm, testX_norm = trainX, testX
    trainY_norm, testY_norm = trainY, testY
    scaler_X, scaler_Y = None, None  # Scaler не используется

# Преобразование в тензоры PyTorch
trainX_norm = torch.tensor(trainX_norm, dtype=torch.float32)
trainY_norm = torch.tensor(trainY_norm, dtype=torch.float32)
testX_norm = torch.tensor(testX_norm, dtype=torch.float32)
testY_norm = torch.tensor(testY_norm, dtype=torch.float32)

print(f"TrainX  shape: {trainX_norm.shape}")    # [batch, seq_len, nodes, channels]
print(f"TrainY  shape: {trainY_norm.shape}")    # [batch, pre_len, nodes, channels]
print(f"TestX   shape: {testX_norm.shape}")     # [batch, seq_len, nodes, channels]
print(f"TestY   shape: {testY_norm.shape}")     # [batch, pre_len, nodes, channels]

TrainX  shape: torch.Size([1594, 12, 325, 3])
TrainY  shape: torch.Size([1594, 6, 325])
TestX   shape: torch.Size([386, 12, 325, 3])
TestY   shape: torch.Size([386, 6, 325])


In [18]:
models = {
    # "DFDGCN": DFDGCN,
    # "LSTM": LSTMModel,
    "GRU": GRUModel,
    "GRUEmb": GRUModelEmb,
    "TCN": TCNModel,
    "TCNEmb": TCNModelEmb,
    # "TGCN": TGCN,
    # "TGCNEmb": TGCNEmb,
    # "STConv": STConvModel,
    # "DCRNN": DCRNNModel
}

param_grids = {
    "LSTM": {
        "hidden_size": [100],
        "num_layers": [2],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32]
    },
    "GRU": {
        "hidden_size": [32],
        "num_layers": [4],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32]
    },
    "GRUEmb": {
        "hidden_size": [32],
        "num_layers": [4],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32],
        "emb_dim": [12]
    },
    "TCN": {
        "num_channels": [32],
        "kernel_size": [3],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32]
    },
    "TCNEmb": {
        "num_channels": [32],
        "kernel_size": [3],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32],
        "emb_dim": [12]
    },
    "STConv": {
        "hidden_channels": [32],
        "kernel_size": [3],
        "lr": [0.005],
        "K": [1],
        "epochs": [30],
        "batch_size": [32]
    },
    "DFDGCN": {
        "supports": [[A]],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32]
    },
    "TGCN": {
        "hidden_dim": [32],
        'num_layers': [2],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32]
    },
    "TGCNEmb": {
        "hidden_dim": [32],
        'num_layers': [2],
        "lr": [0.005],
        "epochs": [30],
        "batch_size": [32],
        "emb_dim": [12]
    }
    # "DCRNN": {
    #     "hidden_channels" : [32],
    #     "num_layers" : [2],
    #     "K" : [2],
    #     "lr": [0.005],
    #     "epochs": [30],
    #     "batch_size": [32]
    # }
}

# Запуск Grid Search
run_grid_search(models, param_grids, trainX_norm, trainY_norm, testX_norm, testY_norm, scaler_Y=scaler_Y, edge_index=edge_index)

All results:
model    params                                                                                                   rmse        mae         r2         var    training_time
-------  --------------------------------------------------------------------------------------------------  ---------  ---------  ---------  ----------  ---------------
GRU      {'batch_size': 32, 'epochs': 30, 'hidden_size': 32, 'lr': 0.005, 'num_layers': 4}                   0.0840138  0.0487244   0.116078  0.00646248         127.458
GRUEmb   {'batch_size': 32, 'emb_dim': 12, 'epochs': 30, 'hidden_size': 32, 'lr': 0.005, 'num_layers': 4}    0.103417   0.0728306  -0.339358  0.00894154         134.922
TCN      {'batch_size': 32, 'epochs': 30, 'kernel_size': 3, 'lr': 0.005, 'num_channels': 32}                 0.0383996  0.0203094   0.815343  0.00147007          73.7434
TCNEmb   {'batch_size': 32, 'emb_dim': 12, 'epochs': 30, 'kernel_size': 3, 'lr': 0.005, 'num_channels': 32}  0.29007    0.232188   -9.53706

In [14]:
run_grid_search(models, param_grids, trainX_norm, trainY_norm, testX_norm, testY_norm, scaler_Y, edge_index)

All results:
model    params                                                                                                  rmse      mae         r2        var    training_time
-------  --------------------------------------------------------------------------------------------------  --------  -------  ---------  ---------  ---------------
GRU      {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                   11.148    6.43265  -1.29221   120.851            139.619
GRUEmb   {'batch_size': 32, 'emb_dim': 12, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}     8.08585  4.63586  -0.205896   64.7468           355.726
TCNEmb   {'batch_size': 32, 'emb_dim': 12, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}   2.5702   1.35783   0.878159    6.34032          352.116
TCN      {'batch_size': 32, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}                  2.56459  1.31859   0.87869     6.57532         

In [ ]:
# All results:
# model    params                                                                                                rmse      mae        r2      var    training_time
# -------  -------------------------------------------------------------------------------------------------  -------  -------  --------  -------  ---------------
# TCNEmb   {'batch_size': 32, 'emb_dim': 4, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}  2.49434  1.29397  0.885246  6.21426          349.874
# TCN      {'batch_size': 32, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}                2.58688  1.3467   0.876573  6.63827          115.958

In [ ]:
# TGCN Epoch [1/50]:   0%|          | 0/50 [00:00<?, ?it/s]                                                                                 All results:
# model    params                                                                               rmse      mae        r2      var    training_time
# -------  --------------------------------------------------------------------------------  -------  -------  --------  -------  ---------------
# TGCN     {'batch_size': 32, 'epochs': 50, 'hidden_dim': 32, 'lr': 0.002, 'num_layers': 2}  126.655  100.314  0.266563  16037.5          231.727

In [12]:
# All results:
# model    params                                                                                                  rmse      mae        r2       var    training_time
# -------  --------------------------------------------------------------------------------------------------  --------  -------  --------  --------  ---------------
# DFDGCN   {'batch_size': 32, 'epochs': 30, 'lr': 0.002, 'supports': [tensor([[0., 1., 0.,  ..., 0., 0., 1.],   33.8138  22.3837  0.947723   1120.15         1645.2
#                  [1., 0., 0.,  ..., 0., 0., 0.],
#                  [0., 0., 0.,  ..., 0., 0., 0.],
#                  ...,
#                  [0., 0., 0.,  ..., 0., 1., 0.],
#                  [0., 0., 0.,  ..., 1., 0., 0.],
#                  [1., 0., 0.,  ..., 0., 0., 0.]])]}
# LSTM     {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                   121.732   95.3205  0.322463  14724.4           154.29
# GRU      {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                   120.737   94.3762  0.3335    14568.4           317.023
# TCN      {'batch_size': 32, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}                  42.8252  28.1331  0.916147   1830.15          128.45

# All results:
# model    params                                                                                                                         rmse      mae        r2      var    training_time
# -------  --------------------------------------------------------------------------------------------------------------------------  -------  -------  --------  -------  ---------------
# DFDGCN   {'batch_size': 32, 'epochs': 30, 'lr': 0.002, 'supports': [tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],  1.98562  1.09549  0.927607   3.9088         1574.38
#                  [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
#                  [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
#                  ...,
#                  [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.9605, 0.6061],
#                  [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.7730],
#                  [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])]}
# LSTM     {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                                           6.7872   4.06529  0.154162  42.553           141.494
# GRU      {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                                           6.61048  3.92131  0.197637  43.5846          139.021
# TCN      {'batch_size': 32, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}                                         2.32921  1.28585  0.900385   5.4208          112.511

# All results:
# model    params                                                                                                                         rmse      mae        r2       var    training_time
# -------  --------------------------------------------------------------------------------------------------------------------------  -------  -------  --------  --------  ---------------
# DFDGCN   {'batch_size': 32, 'epochs': 30, 'lr': 0.002, 'supports': [tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],  2.08501  1.25713  0.920178   3.93405         1576.2
#                  [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
#                  [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
#                  ...,
#                  [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.9605, 0.6061],
#                  [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.7730],
#                  [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])]}
# LSTM     {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                                           6.56797  3.92606  0.207922  41.9298           141.443
# GRU      {'batch_size': 32, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}                                           6.45581  3.84676  0.234744  41.4944           138.62
# TCN      {'batch_size': 32, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}                                         2.35675  1.27149  0.898016   5.50779          114.759

In [13]:
# All results:
# model    params                                                                                  rmse      mae        r2       var
# -------  -----------------------------------------------------------------------------------  -------  -------  --------  --------
# LSTM     {'batch_size': 64, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}    6.83275  3.89527  0.142772  43.9322
# GRU      {'batch_size': 64, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}    6.55248  3.87363  0.211654  42.8969
# TCN      {'batch_size': 64, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}  2.29599  1.28208  0.903207   5.26139

# All results:
# model    params                                                                                  rmse      mae        r2       var
# -------  -----------------------------------------------------------------------------------  -------  -------  --------  --------
# LSTM     {'batch_size': 64, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}    6.65039  3.93296  0.187919  44.2228
# GRU      {'batch_size': 64, 'epochs': 50, 'hidden_size': 32, 'lr': 0.002, 'num_layers': 4}    6.66022  3.94188  0.185517  44.3371
# TCN      {'batch_size': 64, 'epochs': 50, 'kernel_size': 3, 'lr': 0.002, 'num_channels': 32}  2.41745  1.39945  0.892695   5.53997

